<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/Grad_CAMTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for Grad-CAM
Note: Need to add our own trained model first for it to work.

what it does?


1.   Visiulases how the CNN is making certain decisions
2.   It will highlight which areas of our image it is taking into account to make its decision.



In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load your trained autonomous car model
model = tf.keras.models.load_model("your_model.h5")

# Function to preprocess input image
def preprocess_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Grad-CAM function
def compute_gradcam(model, img_array, layer_name="conv2d"):
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions  # Assuming regression (steering angle)

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = np.maximum(heatmap[0], 0)  # ReLU activation
    heatmap /= np.max(heatmap)  # Normalize between 0 and 1
    return heatmap

# Overlay Grad-CAM heatmap on original image
def overlay_heatmap(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)  # Convert to 0-255 scale
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply color map
    overlayed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)

    return overlayed_img

# Load an example image from your dataset
image_path = "example_car_frame.jpg"  # Change to an actual image path
img_array = preprocess_image(image_path)

# Compute Grad-CAM heatmap
heatmap = compute_gradcam(model, img_array, layer_name="conv2d")  # Change "conv2d" to your CNN layer

# Overlay the heatmap on the original image
overlayed_image = overlay_heatmap(image_path, heatmap)

# Display the results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.imread(image_path)[:, :, ::-1])
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(overlayed_image)
plt.title("Grad-CAM Overlay")

plt.show()
